In [1]:
import numpy as np
import pandas as pd

import datetime
import time

import backtrader as bt
from backtrader.feeds import PandasData

%matplotlib inline

In [2]:
class PandasData_Signal(PandasData):
    lines = ('signal',)
    params = (('signal', 6), )

class TestStrategy(bt.Strategy):
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s,%s' % (dt.isoformat(),txt))
        
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datasignal = self.datas[0].signal

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
    def next(self):
        if self.datasignal[0] > 0.7:
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            print(self.log('Current Cash {}'.format(self.broker.get_cash())))
            self.order = self.buy(size=1000)
            
        if self.datasignal[0] < 0.3:
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            self.order = self.sell()


class TestsingleStrategy():
    def __init__(self):
        self.df = pd.read_csv('BackTest/tests/btc_test.csv')
        
        #because our file has like this as int
        self.df['Date'] = [int(time.mktime(datetime.datetime.strptime(x, '%m/%d/%Y').timetuple())) for x in self.df['Date']] 
        #convert timestamp to date
        self.df['Datetime'] = [datetime.datetime.fromtimestamp(x) for x in self.df['Date']]
        
        self.df = self.df.drop('Date', axis=1)
        
        np.random.seed(1)
        self.df['Signal'] = np.random.rand(self.df.shape[0])
        
        self.df = self.df[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Signal']]
        
    def test_Strategy(self):
        cerebro = bt.Cerebro()
        cerebro.broker.setcash(10000)
        cerebro.broker.setcommission(commission=0.001) # 0.1%
        
        print('Starting Portfolio Value: %.2f\n' % cerebro.broker.getvalue())
        
        data = PandasData_Signal(dataname=self.df, datetime=0, open=1, high=2, low=3, close=4, volume=5, signal=6)
        
        cerebro.adddata(data)
        cerebro.addstrategy(TestStrategy)
        
        cerebro.run()
        
        print('\nFinal Portfolio Value: %.2f\n'  % cerebro.broker.getvalue())
        
        cerebro.plot(iplot=False)
        
        
TestsingleStrategy().test_Strategy()

FileNotFoundError: File b'BackTest/tests/btc_test.csv' does not exist